# Two-Dimensional Laplacian Convolutional Representation (LCR-2D)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])
def compute_mae(var, var_hat):
    return np.sum(np.abs(var - var_hat)) / var.shape[0]
def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp1 = np.fft.fft2(lmbda * z - w) / denominator
    temp2 = 1 - N * T / (denominator * np.abs(temp1))
    temp2[temp2 <= 0] = 0
    return np.fft.ifft2(temp1 * temp2).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def flip_mat(X):
    temp = np.append(X, np.flip(X, axis = 1), axis = 1)
    return np.append(temp, np.flip(temp, axis = 0), axis = 0)

def inv_flip_mat(mat):
    dim = mat.shape
    N = int(dim[0] / 2)
    T = int(dim[1] / 2)
    temp = (mat[:, : T] + np.flip(mat[:, T :], axis = 1)) / 2
    return (temp[: N, :] + np.flip(temp[N :, :], axis = 0)) / 2

def LCR_2d(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 10 * lmbda
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    flip_y_true = flip_mat(y_true)
    flip_y = flip_mat(y)
    N, T = flip_y.shape
    pos_train = np.where(flip_y != 0)
    y_train = flip_y[pos_train]
    z = flip_y.copy()
    w = flip_y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    ell = np.fft.fft2(np.outer(ell_s, ell_t))
    denominator = lmbda + gamma * np.abs(ell) ** 2
    del y_true, y
    show_iter = 1
    for it in range(maxiter):
        x = prox_2d(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        y_hat = inv_flip_mat(x)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, y_hat[pos_test]))
            print(compute_rmse(y_test, y_hat[pos_test]))
            print()
    return y_hat

def LCR_3d(y_true, y, lmbda, gamma, tau_s, tau_t, maxiter = 50):
    eta = 10*lmbda
    minRMSE = 100
    minMAE = 100
    flagRMSE = False
    flagMAE = False
    minRMSERound = 0
    minMAERound = 0
    every_round_rmse = []
    every_round_mae = []
    every_round_rmse = [0.0] * (maxiter + 1)
    every_round_mae =[0.0] * (maxiter + 1)
    every_round_rmse[0] = minRMSE
    every_round_mae [0]= minMAE
    endRMSETime=0
    endMAETime=0
    dim1, dim2, dim3 = y.shape
    
    if np.isnan(y).any() == False:
        pos_test = np.where((y_true != 0) & (y == 0))
    elif np.isnan(y).any() == True:
        pos_test = np.where((y_true > 0) & (np.isnan(y)))
        y[np.isnan(y)] = 0
    y_test = y_true[pos_test]
    flip_y_true = np.zeros((2 * dim1, 2 * dim2, dim3))
    flip_y = np.zeros((2 * dim1, 2 * dim2, dim3))
    for t in range(dim3):
        flip_y_true[:, :, t] = flip_mat(y_true[:, :, t])
        flip_y[:, :, t] = flip_mat(y[:, :, t])
    N, T, L = flip_y.shape
    pos_train = np.where(flip_y != 0)
    y_train = flip_y[pos_train]
    z = flip_y.copy()
    w = flip_y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    # ell_s = laplacian(N, tau_s)
    ell_t = laplacian(T, tau_t)
    ell = np.fft.fft2(np.outer(ell_s, ell_t))
    denominator = lmbda + gamma * np.abs(ell) ** 2
    del y_true, y
    show_iter = 10
    start_time=time.time()
    for it in range(maxiter+1):
        x = np.zeros((N, T, L))
        for t in range(dim3):
            x[:, :, t] = prox_2d(z[:, :, t], w[:, :, t], lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        y_hat = np.zeros((dim1, dim2, dim3))
        for t in range(dim3):
            y_hat[:, :, t] = inv_flip_mat(x[:, :, t])
        rmse=compute_rmse(y_test, y_hat[pos_test])
        mae=compute_mae(y_test, y_hat[pos_test])
        every_round_rmse[it]=rmse
        every_round_mae[it]=mae
        print(it + 1,rmse,mae,sep=',')
        if every_round_rmse[it-1]-every_round_rmse[it]>1e-5:
            if minRMSE > every_round_rmse[it]:
                minRMSE = every_round_rmse[it]
                minRMSERound = it
                endRMSETime=time.time()
        else:
            if it-minRMSERound>5:
                flagRMSE=True
                if flagMAE:
                    break
        if every_round_mae[it-1]-every_round_mae[it] > 1e-5:
            if minMAE > every_round_mae[it]:
                minMAE = every_round_mae[it]
                minMAERound = it
                endMAETime = time.time()
        else:
            if it - minMAERound > 5:
                flagMAE = True
                if flagRMSE:
                    break
    print(f"minRMSE:{minRMSE}   minRMSERound:{minRMSERound}    RMSETime:{endRMSETime - start_time}s")
    print(f"minMAE:{minMAE}   minRMSERound:{minMAERound}    MAETime:{endMAETime - start_time}s")
    return y_hat

In [7]:
import numpy as np

# 假设数据存储在名为 'data.txt' 的文件中
trainname = '../datasets/Guangzhou/Guangzhou.txt'

# 初始化一个空列表来存储解析后的数据
full_points = []

# 读取文件并解析每一行
with open(trainname, 'r') as file:
    for line in file:
        # 分割每一行的值
        values = line.strip().split()
        if len(values) >= 4:
            # 将字符串转换为整数或浮点数
            row, col, height = int(values[0]), int(values[1]), int(values[2])
            value = float(values[3])
            # 将解析后的数据添加到列表中
            full_points.append((row, col, height, value))

# 确定张量的最大维度
max_row = 214
max_col = 144
max_height = 61

# 创建一个全零张量

dense_tensor = np.full((max_row + 1, max_col + 1, max_height + 1), np.nan)
# 将解析后的数据填充到张量中
for row, col, height, value in full_points:
         dense_tensor[row, col, height] = value

# 打印张量以验证结果
# 假设数据存储在名为 'data.txt' 的文件中
testname = '../datasets/Guangzhou/Guangzhou_train70.txt'

# 初始化一个空列表来存储解析后的数据
test_points = []

# 读取文件并解析每一行
with open(testname, 'r') as file:
    for line in file:
        # 分割每一行的值
        values = line.strip().split()
        # 将字符串转换为整数或浮点数
        row, col, height = int(values[0]), int(values[1]), int(values[2])
        value = float(values[3])
        # 将解析后的数据添加到列表中
        test_points.append((row, col, height, value))

# 创建一个全零张量
sparse_tensor= np.full((max_row + 1, max_col + 1, max_height + 1), np.nan)
# 将解析后的数据填充到张量中
for row, col, height, value in test_points:
    sparse_tensor[row, col, height] = value

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
import time
start = time.time()
N, T, L = sparse_tensor.shape
lmbda = 1e-3 * N * T
gamma = 1e-6 * lmbda
tau_s = 1
tau_t =61
maxiter = 1000
# dense_tensor = dense_tensor.transpose(1, 0, 2)
# sparse_tensor = densen_tensor.transpose(1, 0, 2)
tensor_hat = LCR_3d(dense_tensor, sparse_tensor, lmbda, gamma, tau_s, tau_t, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))


C:\Users\XXX\AppData\Local\Temp\ipykernel_1236\3108009735.py:21: RuntimeWarning: divide by zero encountered in divide
  temp2 = 1 - N * T / (denominator * np.abs(temp1))


1,39.37721570021934,37.86924717686802
2,38.13974017046122,36.47464838727115
3,37.069103082724396,35.180456154085
4,36.089006821100625,34.02260838885749
5,35.152782131041896,32.96512836664559
6,34.24958328520704,31.97447521982079
7,33.37245024103111,31.02587592579445
8,32.51673690533867,30.107726111699087
9,31.675633172724673,29.213297820853622
10,30.849229269780547,28.340954113201136
11,30.037691305863554,27.4898503943795
12,29.23798926075721,26.65602495704105
13,28.450965285069575,25.84115682377655
14,27.675529165800544,25.044267184628225
15,26.912028859837083,24.26347603185385
16,26.162242122133712,23.50012055515019
17,25.424500020767496,22.754369859075684
18,24.699082997371296,22.025651293846362
19,23.985652458301942,21.312290788257926
20,23.28506771500466,20.61596600747772
21,22.598220825196126,19.935254509919584
22,21.92374317289245,19.269841033250344
23,21.262848049821944,18.620534458183563
24,20.6157808467055,17.987859562466863
25,19.98236177707161,17.372356223282242
26,19.36348